<a href="https://colab.research.google.com/github/Disco-Gnome/CSC84030_Fall23/blob/main/6_pagerank_RGM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

First of all, we authenticate a Google Drive client to download the dataset we will be processing in this Colab.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1EoolSK32_U74I4FeLox88iuUB_SUUYsI'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('web-Stanford.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading

For this Colab we will be using [NetworkX](https://networkx.github.io), a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

The dataset we will analyze is a snapshot of the Web Graph centered around [stanford.edu](https://stanford.edu), collected in 2002. Nodes represent pages from Stanford University (stanford.edu) and directed edges represent hyperlinks between them. [[More Info]](http://snap.stanford.edu/data/web-Stanford.html)

In [6]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

In [7]:
#print(nx.info(G))
#this function has been deprecated since 2008.
#https://networkx.org/documentation/stable/release/api_0.99.html#info

In [8]:
print(G)
print("Number of nodes: ", len(G.nodes))
print("Number of edges: ", len(G.edges))

DiGraph with 281903 nodes and 2312497 edges
Number of nodes:  281903
Number of edges:  2312497


### Your Task

To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.

Use NetworkX to identify the **largest** weakly connected component in the ```G``` graph.  From now on, use this connected component for all the following tasks.

In [65]:
# YOUR CODE HERE
largest_cc = max(nx.weakly_connected_components(G), key=len)
print("Largest CC: ", largest_cc)

Largest CC:  {'228145', '33155', '184138', '112696', '208958', '271444', '124647', '232914', '8347', '98406', '118325', '77751', '101893', '224669', '232624', '17402', '64734', '184250', '250994', '243314', '175208', '237981', '127208', '34792', '2476', '144780', '67764', '224743', '172818', '19363', '107710', '246649', '217435', '222376', '74085', '705', '96468', '152051', '108230', '173955', '35435', '123143', '217548', '224040', '208048', '118048', '254636', '44846', '66234', '191359', '264769', '123005', '152507', '27386', '151647', '260382', '233520', '126858', '203195', '236818', '31596', '236608', '94302', '70901', '21548', '114616', '258131', '69601', '86177', '249898', '71050', '164315', '107831', '97700', '151401', '32222', '24028', '78077', '52914', '57734', '30962', '36016', '34538', '89538', '36400', '85120', '19106', '122960', '133521', '232542', '44304', '216516', '125872', '157987', '233618', '105730', '108964', '165798', '32903', '236364', '216553', '59058', '88994', '

Compute the PageRank vector, using the default parameters in NetworkX: [https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [66]:
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
# YOUR CODE HERE
G_pr = pagerank(G)

In 1999, Barabási and Albert proposed an elegant mathematical model which can generate graphs with topological properties similar to the Web Graph (also called Scale-free Networks).

If you complete the steps below, you should obtain some empirical evidence that the Random Graph model is inferior compared to the Barabási–Albert model when it comes to generating a graph resembling the World Wide Web!

As such, we will use two different graph generator methods, and then we will test how well they approximate the Web Graph structure by means of comparing the respective PageRank vectors. [[NetworkX Graph generators]](https://networkx.github.io/documentation/stable/reference/generators.html#)

Using for both methods ```seed = 1```, generate:


1.   a random graph (with the fast method), setting ```n``` equal to the number of nodes in the original connected component, and ```p = 0.00008```
2.   a Barabasi-Albert graph (with the standard method), setting ```n``` equal to the number of nodes in the original connected component, and finding the right ***integer*** value for ```m``` such as the resulting number of edges **approximates by excess** the number of edges in the original connected component

and compute the PageRank vectors for both graphs.


In [67]:
from networkx.generators.random_graphs import fast_gnp_random_graph, barabasi_albert_graph
from math import ceil

seed = 1
p = 0.00008
n = len(G.nodes)
m = ceil(len(G.edges)/len(G.nodes))

rg_graph = fast_gnp_random_graph(p=p,
                                                 n=n,
                                                 seed=seed)

ba_graph = barabasi_albert_graph(n=n,
                                                              m=m,
                                                              seed=seed)


In [68]:
rg_pr = pagerank(rg_graph)
ba_pr = pagerank(ba_graph)

Compare the PageRank vectors obtained on the generated graphs with the PageRank vector you computed on the original connected component.
**Sort** the components of each vector by value, and use cosine similarity as similarity measure.

Feel free to use any implementation of the cosine similarity available in third-party libraries, or implement your own with ```numpy```.

In [69]:
sorted_G_pr = sorted(G_pr.values())
sorted_rg_pr = sorted(rg_pr.values())
sorted_ba_pr = sorted(ba_pr.values())

In [71]:
from numpy.linalg import norm

a = np.array(sorted_G_pr)
b = np.array(sorted_rg_pr)
cos_sim_rg = a @ b.T / (norm(a)*norm(b))
print("Random graph CosSim: ", cos_sim_rg)

c = np.array(sorted_ba_pr)
cos_sim_ba = a @ c.T/(norm(a)*norm(c))
print("Barabasi-Albert graph CosSim: ", cos_sim_ba)

Random graph CosSim:  0.10655288123733446
Barabasi-Albert graph CosSim:  0.6556823685587775


#### **Submission Intruction:**

#### Click File -> Download -> Download **.ipynb**, and upload the downloaded file to Blackboard.